<a href="https://colab.research.google.com/github/AbhishekMajhi/DeepLearning-Using-Tensorflow/blob/master/RNN/Sentence_Embeddings_With_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install simpletransformers

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(48)
# Import libraries

from time import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


import itertools
import datetime

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda,Dense,BatchNormalization, Flatten, Dropout, Activation
from tensorflow.keras.regularizers import l2
import keras.backend as K
from tensorflow import keras
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform


2021-07-20 04:28:43.407324: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import csv
def read_csv(filename = '/content/train_command.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y


In [ ]:
X_train, Y_train = read_csv('/content/train_command.csv')
X_test, Y_test= read_csv('/content/validation command.csv')

In [ ]:
X_train.shape

(167,)

## BERT Model

In [ ]:
from simpletransformers.language_representation import RepresentationModel

In [ ]:
# Initializing BERT Model
model_bert = RepresentationModel(model_type= 'bert',
                            model_name= 'bert-base-uncased',
                            use_cuda= False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_sentence_vec = model_bert.encode_sentences(X_train,combine_strategy="mean")
val_sentence_vec = model_bert.encode_sentences(X_test,combine_strategy="mean")

In [ ]:
print("train_vec:",train_sentence_vec.shape)
print("val_vec:",val_sentence_vec.shape)

train_vec: (167, 768)
val_vec: (43, 768)


In [ ]:
maxLen = len(max(X_train, key=len).split())
maxLen

15

In [ ]:
# one-hot encode
Y_train_oh = tf.keras.utils.to_categorical(Y_train,num_classes = 23)
Y_test_oh = tf.keras.utils.to_categorical(Y_test,num_classes = 23)

In [ ]:
train_sentence_vec.shape

(167, 768)

In [ ]:
Y_train_oh.shape

(167, 23)

## Similarity Model

In [ ]:
### temp code for exper..###
def similarity_model(input_shape):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    embeddings = tf.keras.layers.Input(input_shape,dtype = 'float32')
    
    # # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    # embedding_layer = pretrained_embedding_layer(word_to_vec_map,word_to_index)
    
    # # Propagate sentence_indices through your embedding layer
    # # (See additional hints in the instructions).
    # embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = tf.keras.layers.LSTM(128,return_sequences =True, name = 'hidden1')(embeddings)
    X = tf.keras.layers.BatchNormalization()(X)
    # Add dropout with a probability of 0.5
    X = tf.keras.layers.Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = tf.keras.layers.LSTM(128,return_sequences = True)(X)
    X = tf.keras.layers.BatchNormalization()(X)
    # Add dropout with a probability of 0.5
    X = tf.keras.layers.Dropout(0.5)(X)
    # X = LSTM(128,return_sequences = True,activity_regularizer= l2(0.03))(X)
    # X = BatchNormalization()(X)
    # X = Dropout(0.5)(X)
    X = tf.keras.layers.LSTM(64,return_sequences = False)(X)
    #X = Flatten()(X)
    # Propagate X through a Dense layer with 5 units
    X = tf.keras.layers.Dense(units=23)(X)
    # Add a softmax activation
    X = tf.keras.layers.Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = tf.keras.models.Model(embeddings,X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
train_sentence_vec.shape

(167, 768)

In [ ]:
Y_train_oh.shape

(167, 23)

In [ ]:
model = similarity_model((train_sentence_vec.shape[1],1))
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 768, 1)]          0         
_________________________________________________________________
hidden1 (LSTM)               (None, 768, 128)          66560     
_________________________________________________________________
batch_normalization_2 (Batch (None, 768, 128)          512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 768, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 768, 128)          131584    
_________________________________________________________________
batch_normalization_3 (Batch (None, 768, 128)          512       
_________________________________________________________________
dropout_3 (Dropout)          (None, 768, 128)          0   

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(train_sentence_vec.shape)
print(Y_train_oh.shape)

(167, 768)
(167, 23)


In [ ]:
history = model.fit(train_sentence_vec, Y_train_oh, epochs = 1000, batch_size = 30, shuffle=True,validation_data=(val_sentence_vec, Y_test_oh))

In [ ]:
sen = np.array(['what will be the weather today?'])
emb = model_bert.encode_sentences(sen,combine_strategy="mean")

In [ ]:
print(np.argmax(model.predict(emb)))

15
